In [1]:
# Import dependencies
import datetime
import dateutil
from openclsim.appendix import get_event_log
import openclsim.model as model
import openclsim.plugins as plugins
import simpy
import numpy as np

In [2]:
# Create a simulation environment
class SimulationEnvironment(object):
    """
    A OpenCLSim simulation object.

    Use the class methods to define the activities and start the
    simulation. In this example, a basic activity is modelled using 
    the `model.BasicActivity` class and the length of the activity is
    extended using a random delay with the `model.HasDelayPlugin`. For
    further reference, check the `DelayBasicActivity` class below.

    Parameters
    ----------
        start_date: datetime.datetime
            A datetime object specifying the simulation start date.
    """
    def __init__(self, start_date: datetime.datetime):

        # Define start date and time of simulation
        self.start_date = start_date
        start_utc = start_date.replace(tzinfo=dateutil.tz.UTC)
        start_epoch = start_utc.timestamp()

        # Initialise a SimPy simulation environment
        self.env = simpy.Environment(initial_time=start_epoch)
        self.registry = {}

    def define_activities(self):
        """Use this function to define your activities."""

        # Register the activities
        model.register_processes([...]) 
        
        # Return the base activity for logging purposes
        return [activity]  
        
    def execute_simulation(self):
        """Function starts the simulation."""
        self.activities = self.define_activities()
        self.env.run(until=self.env.now + 60 * 61) # Loop till an hour has passed.
        return 'SUCCESSFUL'

    @property
    def event_log(self):
        """Function returns the event log."""
        return get_event_log(activity_list=self.activities)

    @property
    def project_length(self):
        """Function returns the project length."""
        log = self.event_log
        dt = (log.Timestamp.iloc[-1] - log.Timestamp.iloc[0])
        return dt.total_seconds() / (3600)

In [6]:
class BasicActivity(plugins.HasRequestWindowPluginActivity, model.BasicActivity):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

# Create Environment
start_date = datetime.datetime(2021, 1, 1)
start_utc = start_date.replace(tzinfo=dateutil.tz.UTC)
start_epoch = start_utc.timestamp()
env = simpy.Environment(initial_time=start_epoch)
registry = {}

# Create offshore env
oe = plugins.OffshoreEnvironment()
oe.store_information(var='hs', filename='test.csv')
oe.store_information(var='tp', filename='test.csv')

# Define activity
activity = BasicActivity(
    env=env, 
    registry=registry,
    name="A basic activity",
    limit_expr=lambda hs, tp: hs > 0.5,
    offshore_environment=oe,
    duration=3600
)

# Register activity
model.register_processes([activity])

# Run sim
env.run()

In [7]:
get_event_log(activity_list=[activity])

,Timestamp,ActivityState,Description,ActivityID
0,2021-01-01 00:00:00,WAIT_START,A basic activity,42c8f990-cfd9-457a-9a56-e49f9db3a8cc
1,2021-01-01 01:00:00,WAIT_STOP,A basic activity,42c8f990-cfd9-457a-9a56-e49f9db3a8cc
2,2021-01-01 01:00:00,START,A basic activity,42c8f990-cfd9-457a-9a56-e49f9db3a8cc
3,2021-01-01 02:00:00,STOP,A basic activity,42c8f990-cfd9-457a-9a56-e49f9db3a8cc


In [8]:
oe.df

,hs,tp,limit_exceeded,block
date,,,,
2021-01-01,0.551281,0.551281,True,1
2021-01-02,0.405864,0.405864,False,2
2021-01-03,0.415363,0.415363,False,2
2021-01-04,0.041528,0.041528,False,2
2021-01-05,0.444675,0.444675,False,2
...,...,...,...,...
2021-12-27,0.994670,0.994670,True,183
2021-12-28,0.123720,0.123720,False,184
2021-12-29,0.610298,0.610298,True,185
